#### Sample Class (Ignore Me)

In [ ]:
import numpy as np
import copy

class Solution(object):
  def __init__(self, arr, values, weights):
    self.solution = arr
    self.values = values
    self.weights = weights

  def get_value(self):
    return np.matmul(self.solution, self.values)

  def get_weight(self):
    return np.matmul(self.solution, self.weights)

def get_neighbors(solution, values, weights):
  neighbors = Solution([], values, weights)
  for i in range(len(solution)):
    new_neighbor = copy.deepcopy(solution)

    if new_neighbor[i] == 1:
      new_neighbor[i] = 0
    else:
      new_neighbor[i] = 1
    neighbors.append(new_neighbor)

  return neighbors

In [ ]:
values = np.array([89, 90, 30, 50, 90, 79, 90, 10])
weights = np.array([123, 154, 258, 354, 365, 150, 95, 195])
initial_soln = Solution([0, 1, 0, 0, 1, 0, 0, 0], values, weights)

In [ ]:
initial_soln.get_value(), initial_soln.get_weight()

(180, 519)

### Knapsack Problem

In [1]:
# packages
import numpy as np
import copy
import random

#### Helper Functions (run me first)

In [10]:
def get_value(soln, values):
  return np.matmul(soln, values)

def get_weight(soln, weights):
  return np.matmul(soln, weights)

def get_order(limit, minimization=False):
  if minimization:
    return 1
  else:
    return -1

def generate_init_soln(size, values, limit, minimization=False, method='Random'):
  """
  Parameter
  -----------
  size: Number of decision variables
  method: 'Random' or 'Greedy Search'

  Returns
  -----------
  `np.ndarray` initial solution.

  """

  order = get_order(limit, minimization)

  if method == "Random":
    feasibility = False
    while feasibility == False:
      initial_soln = np.random.randint(0, 2, size)
      w = get_weight(initial_soln, weights)
      if order*w >= order*limit:
        feasibility = True

  if method == 'Greedy Search':
    indices = values.argsort()[::order]
    initial_soln = np.zeros(size)
    new_init_soln = initial_soln

    for ind in range(len(indices)):
      new_init_soln = copy.deepcopy(new_init_soln)
      new_init_soln[indices[ind]] = 1
      w = get_weight(new_init_soln, weights)
      if order*w >= order*limit:
        initial_soln[indices[ind]] = 1
      else:
        break

  return initial_soln

def generate_neighbors(soln):
  neighbors = []
  for i in range(len(soln)):
    new_neighbor = copy.deepcopy(soln)
    if new_neighbor[i] == 1:
      new_neighbor[i] = 0
    else:
      new_neighbor[i] = 1
    neighbors.append(new_neighbor)
  return neighbors

def select_best_neighbor(neighbors, values, weights, limit, minimization=False):
  order = get_order(limit, minimization)
  nb_values, nb_weights = get_value(neighbors, values), get_weight(neighbors, weights)


  filtered_inds = np.where(order*nb_weights >= order*limit)[0]
  feasible_nb = [neighbors[i] for i in filtered_inds]

  if minimization:
    best_ind = np.argmin(get_value(feasible_nb, values))
  else:
    best_ind = np.argmax(get_value(feasible_nb, values))

  return feasible_nb[best_ind]

def compare_solns(incumbent_soln, best_nb, values, weights, limit, continue_iter=True):
  order = get_order(limit)
  prev_value, new_value = get_value(incumbent_soln, values), get_value(best_nb, values)

  if order*prev_value <= order*new_value:
    continue_iter = False
  return continue_iter

def optimal_soln(incumbent_soln, labels):
  opt_labels = []
  for i in range(len(incumbent_soln)):
    if incumbent_soln[i] == 1:
      opt_labels.append(labels[i])

  return opt_labels

#### Complete Implementation

In [11]:
# ---- Parameters
numVar = 8
labels = ['wine', 'beer', 'pizza', 'burger', 'fries', 'coke', 'apple', 'donut']
values = np.array([89, 90, 30, 50, 90, 79, 90, 10])
weights = np.array([123, 154, 258, 354, 365, 150, 95, 195])
max_weight = 750
minimization = False

continue_iter = True
iter_count = 0

init_soln = generate_init_soln(size=numVar, values=values, limit=max_weight, method='Random')
print(f'Initial solution: {init_soln} \n value = {get_value(init_soln, values)}, weight = {get_weight(init_soln, weights)} \n')

incumbent_soln = init_soln

while continue_iter:
  print(f'ITERATION {iter_count+1}: --------- \n')

  # generating neighbors
  neighbors = generate_neighbors(incumbent_soln)
  print(f'Neighbors: \n values = {get_value(neighbors, values)} \n weights = {get_weight(neighbors, weights)} \n')

  # selecting best neighbor
  best_nb = select_best_neighbor(neighbors, values, weights, max_weight)
  print(f'Best neighbor: {best_nb} \n value = {get_value(best_nb, values)} \n weight = {get_weight(best_nb, weights)} \n')

  # compare incumbent solution and best neighbor
  continue_iter = compare_solns(incumbent_soln, best_nb, values, weights, max_weight)
  if continue_iter:
    incumbent_soln = best_nb

  print(f'Incumbent solution: {incumbent_soln} \n value = {get_value(incumbent_soln, values)} \n weight = {get_weight(incumbent_soln, weights)} \n')
  iter_count += 1

print('--------------------------------------------')
print(f'After {iter_count} iteration(s), optimal solution to include \n {optimal_soln(incumbent_soln, labels)} \n value = {get_value(incumbent_soln, values)} \n weight = {get_weight(incumbent_soln, weights)} \n')

Initial solution: [0 0 1 0 1 0 0 0] 
 value = 120, weight = 623 

ITERATION 1: --------- 

Neighbors: 
 values = [209 210  90 170  30 199 210 130] 
 weights = [746 777 365 977 258 773 718 818] 

Best neighbor: [0 0 1 0 1 0 1 0] 
 value = 210 
 weight = 718 

Incumbent solution: [0 0 1 0 1 0 1 0] 
 value = 210 
 weight = 718 

ITERATION 2: --------- 

Neighbors: 
 values = [299 300 180 260 120 289 120 220] 
 weights = [ 841  872  460 1072  353  868  623  913] 

Best neighbor: [0 0 0 0 1 0 1 0] 
 value = 180 
 weight = 460 

Incumbent solution: [0 0 1 0 1 0 1 0] 
 value = 210 
 weight = 718 

--------------------------------------------
After 2 iteration(s), optimal solution to include 
 ['pizza', 'fries', 'apple'] 
 value = 210 
 weight = 718 

